In [73]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_colwidth', None)

In [3]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Step 1: Scrape listing links
i = 4258
url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
houselinks = []
while True:  # Adjust the range as needed
    i += 1
    r = requests.get(url, headers=headers)

    url = f'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/cala-polska?page={i}'
    print(f'Page {i} status code:', r.status_code)
    soup = BeautifulSoup(r.content, 'lxml')

    # Check if no listings found
    h3_element = soup.find('h3', class_='css-1nw1os0 e1ws6l2x3')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found. Stopping the scraping process.")
        break

    listing_container = soup.find_all('section', class_='eeungyz1 css-hqx1d9 e12fn6ie0')
    if not listing_container:
        print(f"No listing container found on page {i}")
    else:
        print(f"Found {len(listing_container)} listing containers on page {i}")

    # Extract individual listing URLs
    for section in listing_container:
        links = section.find_all('a', class_='css-16vl3c1 e17g0c820')
        for link in links:
            href = link['href']
            if href and href not in houselinks:
                # Ensure the URL is complete
                if not href.startswith('http'):
                    href = 'https://www.otodom.pl' + href
                houselinks.append(href)

print(f"Total houselinks found: {len(houselinks)}")

# Step 2: Scrape details from each listing
houselist = []
for link in houselinks:
    r = requests.get(link, headers=headers)
    if r.status_code == 403:
        print(f"Access denied for link: {link}")
        continue

    soup = BeautifulSoup(r.text, 'lxml')
    body = soup.select_one('main')
    if not body:
        print(f"No main section found for {link}")
        continue

    # Check if no listings found
    h3_element = body.find('h3', class_='css-1nw1os0 e1ysrxc23')
    if h3_element and h3_element.get_text(strip=True) == 'Nie znaleźliśmy żadnych ogłoszeń':
        print("No advertisements found on this detail page. Stopping the scraping process.")
        break

    try:
        NumerId = link

        nazwa = body.find('h1', class_='css-1wnihf5 e1j8g12x8')
        nazwa = nazwa.text.strip() if nazwa else "N/A"

        cenaM2 = body.find('div', class_='css-1h1l5lm e1j8g12x9')
        cenaM2 = cenaM2.text.strip() if cenaM2 else "N/A"

        cena = body.find('strong', class_='css-t3wmkv e9aa0kv0')
        cena = cena.text.strip() if cena else "N/A"

        adres = body.find('a', class_='eozeyij0 css-1helwne e1p0dzoz0')
        adres = adres.text.strip() if adres else "N/A"

        powierzchnia = body.find('div', {'aria-label': 'Powierzchnia', 'class': 'css-1ivc1bc e26jmad1'})
        powierzchnia = powierzchnia.text.strip() if powierzchnia else "N/A"

        formaWlasnosci = body.find('div', {'aria-label': 'Forma własności', 'class': 'css-1ivc1bc e26jmad1'})
        formaWlasnosci = formaWlasnosci.text.strip() if formaWlasnosci else "N/A"

        LiczbaPokoi = body.find('div', {'aria-label': 'Liczba pokoi', 'class': 'css-1ivc1bc e26jmad1'})
        LiczbaPokoi = LiczbaPokoi.text.strip() if LiczbaPokoi else "N/A"

        stanWykonczenia = body.find('div', {'aria-label': 'Stan wykończenia','class':'css-1ivc1bc e26jmad1'})
        stanWykonczenia = stanWykonczenia.text.strip() if stanWykonczenia else "N/A"

        pietro = body.find('div', {'aria-label': 'Piętro', 'class': 'css-1ivc1bc e26jmad1'})
        pietro = pietro.text.strip() if pietro else "N/A"

        BalkonOgrodTaras = body.find('div', {'aria-label': 'Balkon / ogród / taras', 'class': 'css-1ivc1bc e26jmad1'})
        BalkonOgrodTaras = BalkonOgrodTaras.text.strip() if BalkonOgrodTaras else "N/A"

        Czynsz = body.find('div', {'aria-label': 'Czynsz', 'class': 'css-1ivc1bc e26jmad1'})
        Czynsz = Czynsz.text.strip() if Czynsz else "N/A"
        
        MiejsceParkingowe = body.find('div', {'aria-label': 'Miejsce parkingowe', 'class': 'css-1ivc1bc e26jmad1'})
        MiejsceParkingowe = MiejsceParkingowe.text.strip() if MiejsceParkingowe else "N/A"
        
        ObslugaZdalna = body.find('div', {'aria-label': 'Obsługa zdalna', 'class': 'css-1ivc1bc e26jmad1'})
        ObslugaZdalna = ObslugaZdalna.text.strip() if ObslugaZdalna else "N/A"

        ogrzewanie = body.find('div', {'aria-label': 'Ogrzewanie', 'class': 'css-1wi2w6s e26jmad5'})
        ogrzewanie = ogrzewanie.text.strip() if ogrzewanie else "N/A"

        Rynek = body.find('div', {'aria-label': 'Rynek', 'class': 'css-tpkder e26jmad1'})
        Rynek = Rynek.text.strip() if Rynek else "N/A"

        TypOgloszeniodawcy = body.find('div', {'aria-label': 'Typ ogłoszeniodawcy', 'class': 'css-tpkder e26jmad1'})
        TypOgloszeniodawcy = TypOgloszeniodawcy.text.strip() if TypOgloszeniodawcy else "N/A"

        DostepneOd = body.find('div', {'aria-label': 'Dostępne od', 'class': 'css-tpkder e26jmad1'})
        DostepneOd = DostepneOd.text.strip() if DostepneOd else "N/A"

        RokBudowy = body.find('div', {'aria-label': 'Rok budowy', 'class': 'css-tpkder e26jmad1'})
        RokBudowy = RokBudowy.text.strip() if RokBudowy else "N/A"

        RodzajZabudowy = body.find('div', {'aria-label': 'Rodzaj zabudowy', 'class': 'css-tpkder e26jmad1'})
        RodzajZabudowy = RodzajZabudowy.text.strip() if RodzajZabudowy else "N/A"
        
        Okna = body.find('div', {'aria-label': 'Okna', 'class': 'css-tpkder e26jmad1'})
        Okna = Okna.text.strip() if Okna else "N/A"

        CzyWinda = body.find('div', {'aria-label': 'Winda', 'class': 'css-tpkder e26jmad1'})
        CzyWinda = CzyWinda.text.strip() if CzyWinda else "N/A"
        
        Media = body.find('div', {'aria-label': 'Media', 'class': 'css-tpkder e26jmad1'})
        Media = Media.text.strip() if Media else "N/A"

        Zabezpieczenia = body.find('div', {'aria-label': 'Zabezpieczenia', 'class': 'css-tpkder e26jmad1'})
        Zabezpieczenia = Zabezpieczenia.text.strip() if Zabezpieczenia else "N/A"

        Wyposazenie = body.find('div', {'aria-label': 'Wyposażenie', 'class': 'css-tpkder e26jmad1'})
        Wyposazenie = Wyposazenie.text.strip() if Wyposazenie else "N/A"

        InformacjeDodatkowe = body.find('div', {'aria-label': 'Informacje dodatkowe', 'class': 'css-tpkder e26jmad1'})
        InformacjeDodatkowe = InformacjeDodatkowe.text.strip() if InformacjeDodatkowe else "N/A"

        MaterialBudynku = body.find('div', {'aria-label': 'Materiał budynku', 'class': 'css-tpkder e26jmad1'})
        MaterialBudynku = MaterialBudynku.text.strip() if MaterialBudynku else "N/A"

        rooms = {
            'ID': NumerId,
            'Nazwa': nazwa,
            'CenaM2': cenaM2,
            'Cena': cena,
            'Adres': adres,
            'Powierzchnia': powierzchnia,
            'Forma_Wlasnosci': formaWlasnosci,
            'LiczbaPokoi': LiczbaPokoi,
            'StanWykonczenia': stanWykonczenia,
            'Pietro': pietro,
            'BalkonOgrodTaras': BalkonOgrodTaras,
            'Czynsz': Czynsz,
            'Ogrzewanie': ogrzewanie,
            'Rynek': Rynek,
            'TypOgloszeniodawcy': TypOgloszeniodawcy,
            'DostepneOd': DostepneOd,
            'RokBudowy': RokBudowy,
            'RodzajZabudowy': RodzajZabudowy,
            'CzyWinda': CzyWinda,
            'Zabezpieczenia': Zabezpieczenia,
            'Wyposazenie': Wyposazenie,
            'InformacjeDodatkowe': InformacjeDodatkowe,
            'MaterialBudynku': MaterialBudynku
        }
        houselist.append(rooms)
    except AttributeError as e:
        #print(f"Error parsing details for {link}: {e}")
        continue

# Save the data to a DataFrame
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(houselist)
df

Page 4259 status code: 200
Found 39 listing containers on page 4259
Page 4260 status code: 200
Found 39 listing containers on page 4260
Page 4261 status code: 200
Found 39 listing containers on page 4261
Page 4262 status code: 200
Found 19 listing containers on page 4262
Page 4263 status code: 200
No advertisements found. Stopping the scraping process.
Total houselinks found: 136


,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,Rynek,TypOgloszeniodawcy,DostepneOd,RokBudowy,RodzajZabudowy,CzyWinda,Zabezpieczenia,Wyposazenie,InformacjeDodatkowe,MaterialBudynku
0,https://www.otodom.pl/pl/oferta/kawalerka-33m-balkon-nowe-budownictwo-premium-ID4hEoS,"Kawalerka 33m Balkon, nowe budownictwo, premium",9 030 zł/m²,298 000 zł,"Skarżysko-Kamienna, skarżyski, świętokrzyskie",Powierzchnia33 m²,Forma własnościpełna własność,Liczba pokoi1,Stan wykończeniado zamieszkania,Piętro4/4,...,Rynekpierwotny,Typ ogłoszeniodawcyprywatny,Dostępne odbrak informacji,Rok budowy2018,Rodzaj zabudowyapartamentowiec,Windatak,"Zabezpieczeniadrzwi / okna antywłamaniowe, teren zamknięty, domofon / wideofon, monitoring / ochrona","Wyposażeniezmywarka, lodówka, meble, piekarnik, kuchenka, telewizor, pralka",Informacje dodatkowebrak informacji,Materiał budynkupustak
1,https://www.otodom.pl/pl/oferta/komfortowy-apartament-sopot-dolny-ID4qxDu,Komfortowy apartament Sopot Dolny,35 070 zł/m²,2 490 000 zł,"ul. Tadeusza Kościuszki, Centrum Południe, Dolny Sopot, Sopot, pomorskie",Powierzchnia71 m²,Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętro2/3,...,Rynekwtórny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne od2024-05-12,Rok budowy2018,Rodzaj zabudowyapartamentowiec,Windatak,"Zabezpieczeniasystem alarmowy, teren zamknięty, domofon / wideofon, monitoring / ochrona","Wyposażeniezmywarka, lodówka, meble, piekarnik, kuchenka, telewizor, pralka","Informacje dodatkoweklimatyzacja, piwnica",Materiał budynkucegła
2,https://www.otodom.pl/pl/oferta/gotowe-mieszkanie-dwustronne-3-pokoje-jagodno-ID4qYhV,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,https://www.otodom.pl/pl/oferta/mieszkanie-sopot-centrum-ID45Iqf,Mieszkanie - Sopot Centrum,12 586 zł/m²,1 100 000 zł,"Górny Sopot, Sopot, pomorskie","Powierzchnia87,4 m²",Forma własnościpełna własność,Liczba pokoi3,Stan wykończeniado zamieszkania,Piętroparter/2,...,Rynekwtórny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne odbrak informacji,Rok budowybrak informacji,Rodzaj zabudowydom wolnostojący,Windanie,Zabezpieczeniabrak informacji,Wyposażeniemeble,Informacje dodatkowebrak informacji,Materiał budynkubrak informacji
4,https://www.otodom.pl/pl/oferta/mieszkanie-2-pokoje-38m2-polesie-ID45HQd,Mieszkanie 2 pokoje 38m2 Polesie,6 959 zł/m²,270 000 zł,"Żubardź, Bałuty, Łódź, łódzkie","Powierzchnia38,8 m²",Forma własnościpełna własność,Liczba pokoi2,Stan wykończeniado remontu,Piętro8/10,...,Rynekwtórny,Typ ogłoszeniodawcybiuro nieruchomości,Dostępne odbrak informacji,Rok budowybrak informacji,Rodzaj zabudowybrak informacji,Windatak,Zabezpieczeniabrak informacji,Wyposażeniebrak informacji,Informacje dodatkowepiwnica,Materiał budynkubrak informacji
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,https://www.otodom.pl/pl/oferta/przetarg-na-sprzedaz-1-2-udzialu-w-mieszkaniu-ID4qFqY,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
132,https://www.otodom.pl/pl/oferta/sprzedam-mieszkanie-ID4qw1q,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
133,https://www.otodom.pl/pl/oferta/zamienie-kawalerke-na-dzialke-budowlana-ID4qqjJ,Zamienię kawalerkę na działkę budowlaną,5 181 zł/m²,100 000 zł,"ul. ks. Piotra Wawrzyniaka, Inowrocław, inowrocławski, kujawsko-pomorskie","Powierzchnia19,3 m²",Forma własnościpełna własność,Liczba pokoi1,Stan wykończeniaZapytaj,Piętroparter,...,Rynekwtórny,Typ ogłoszeniodawcyprywatny,Dostępne odbrak informacji,Rok budowybrak informacji,Rodzaj zabudowybrak informacji,Windanie,Zabezpieczeniabrak informacji,Wyposażeniebrak informacji,Informacje dodatkowebrak informacji,Materiał budynkubrak informacji
134,https://www.otodom.pl/pl/oferta/apartament-sopot-dolny-400-m-do-plazy-i-mola-ID4qkjq,apartament SOPOT dolny 400 m do plaży i mola,21 000 zł/m²,1 050 000 zł,"ul. Fryderyka Chopina, Dol

In [75]:
df_copy = df.copy()

In [76]:
## FEATURE ENGINEERING

In [77]:
## ID
df_copy['ID'] = df_copy['ID'].apply(lambda x: re.search(r'(ID\w+)', x).group(0) if re.search(r'(ID\w+)', x) else None)

In [78]:
# Cena
df_copy['Cena'] = df_copy['Cena'].apply(lambda y: str(y).split('zł')[0])
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace(' ', ''))
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace(',', '.'))
df_copy['Cena'] = df_copy['Cena'].apply(lambda x: x.replace('Zapytajocenę',''))
df_copy['Cena'] = df_copy['Cena'].replace(['N/A', ''], np.nan)
df_copy['Cena'] = df_copy['Cena'].replace('', np.nan)
df_copy['Cena'] = df_copy['Cena'].astype(float)


In [79]:
# CenaM2
df_copy['CenaM2'] = df_copy['CenaM2'].apply(lambda y: str(y).split('zł/m²')[0])
df_copy['CenaM2'] = df_copy['CenaM2'].apply(lambda x: x.replace(' ', ''))
df_copy['CenaM2'] = df_copy['CenaM2'].replace('', np.nan)
df_copy['CenaM2'] = df_copy['CenaM2'].replace(['N/A', ''], np.nan)
df_copy['CenaM2'] = df_copy['CenaM2'].astype(float)

In [80]:
## Nazwa czy potrzebna ?

In [81]:
## Adres
### Wyodrębnienie nazwy wojewodztwa

wojewodztwa = ['dolnośląskie','kujawsko-pomorskie','lubelskie','lubuskie','łódzkie',
               'małopolskie','mazowieckie','opolskie','podkarpackie','podlaskie','pomorskie',
               'śląskie','świętokrzyskie','warmińsko-mazurskie','wielkopolskie','zachodniopomorskie']



In [82]:
df_copy['Wojewodztwo'] = df_copy['Adres'].str.split().apply(lambda x: '_'.join([m for m in x if m in wojewodztwa])).replace('',np.nan)

In [83]:
### Wyodrębnienie nazwy miast
file_name = r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListOfCity\SIMC_15-06-2024 ({}).csv'
df_city_poland = pd.concat([pd.read_csv(file_name.format(i),sep=';') for i in range(0, 16)])

In [84]:
city_set = {city.strip().lower() for city in df_city_poland['NAZWA']}

def extract_city_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    city = next((part for part in parts if part in city_set), np.nan)
    return city.capitalize() if city is not np.nan else city

df_copy['Miasto'] = df_copy['Adres'].apply(extract_city_values)


In [85]:
### Wyodrębnienie nazwy powiatow

In [86]:
df_district_poland = pd.read_excel(r'C:\Users\seba_\Desktop\PythonProject\OtoDom\ListofDistrict\ListaPowiatow.xlsx')

In [87]:
district_set = {district.strip().lower() for district in df_district_poland['Powiat']}

def extract_district_values(address):
    parts = [part.strip().lower() for part in address.split(',')]
    district = next((part for part in parts if part in district_set), np.nan)
    return district.capitalize() if district is not np.nan else district

df_copy['Powiat'] = df_copy['Adres'].apply(extract_district_values)

In [88]:
df_copy[['Miasto','Adres','Powiat','Wojewodztwo']]

,Miasto,Adres,Powiat,Wojewodztwo
0,Skarżysko-kamienna,"Skarżysko-Kamienna, skarżyski, świętokrzyskie",Skarżyski,świętokrzyskie
1,Sopot,"ul. Tadeusza Kościuszki, Centrum Południe, Dolny Sopot, Sopot, pomorskie",Sopot,pomorskie
2,NaN,N/A,NaN,NaN
3,Sopot,"Górny Sopot, Sopot, pomorskie",Sopot,pomorskie
4,Łódź,"Żubardź, Bałuty, Łódź, łódzkie",Łódź,łódzkie
...,...,...,...,...
131,NaN,N/A,NaN,NaN
132,NaN,N/A,NaN,NaN
133,Inowrocław,"ul. ks. Piotra Wawrzyniaka, Inowrocław, inowrocławski, kujawsko-pomorskie",Inowrocławski,kujawsko-pomorskie
134,Sopot,"ul. Fryderyka Chopina, Dolny Sopot - Grunwaldzka, Dolny Sopot, Sopot, pomorskie",Sopot,pomorskie


In [89]:
### Wyodrębnienie nazwy ulicy

In [90]:
def extract_ulica_values(address):
    matches = re.findall(r'\bul\.\s*[^,]*', address)
    return ', '.join(matches) if matches else np.nan

df_copy['Ulica'] = df_copy['Adres'].apply(extract_ulica_values)

In [91]:
##Powierzchnia

df_copy['Powierzchnia'] = df_copy['Powierzchnia'].str.replace('Powierzchnia', '').str.strip()
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].apply(lambda y: str(y).split('m²')[0])
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].apply(lambda x: x.replace(',', '.'))
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].replace(['N/A', ''], np.nan)
df_copy['Powierzchnia'] = df_copy['Powierzchnia'].astype(float)

In [92]:
##Forma_Wlasnosci

df_copy['Forma_Wlasnosci'] = df_copy['Forma_Wlasnosci'].str.replace('Forma własności', '').str.strip()

In [93]:
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].str.replace('Liczba pokoi', '').str.strip()
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].replace(['N/A', 'NaN', ''], np.nan)
df_copy['LiczbaPokoi'] = df_copy['LiczbaPokoi'].fillna(0).astype(int)

In [94]:
##StanWykonczenia
df_copy['StanWykonczenia'] = df_copy['StanWykonczenia'].str.replace('Stan wykończenia', '').str.strip()

In [95]:
##Pietro
df_copy['Pietro'] = df_copy['Pietro'].str.replace('Piętro', '').str.strip()
df_copy['Pietro'] = df_copy['Pietro'].str.replace('parter', '0')
df_copy['Pietro'] = df_copy['Pietro'].replace('poddasze', '100')
df_copy['Pietro'] = df_copy['Pietro'].replace('Zapytaj', '')
df_copy['Pietro'] = df_copy['Pietro'].replace('>', '')
df_copy['Pietro'] = df_copy['Pietro'].replace('', np.nan)
df_copy['Pietro'] = df_copy['Pietro'].apply(lambda x: int(x.split('/')[0]) if isinstance(x, str) and '/' in x else x)


ValueError: invalid literal for int() with base 10: 'N'

In [96]:
##RodzajZabudowy
df_copy['RodzajZabudowy'] = df_copy['RodzajZabudowy'].str.replace('Rodzaj zabudowy', '').str.strip()

In [97]:
##CzyWinda
df_copy['CzyWinda'] = df_copy['CzyWinda'].str.replace('Winda', '').str.strip()

In [98]:
##Zabezpieczenia
df_copy['Zabezpieczenia'] = df_copy['Zabezpieczenia'].str.replace('Zabezpieczenia', '').str.strip()

In [99]:
##Wyposazenie
df_copy['Wyposazenie'] = df_copy['Wyposazenie'].str.replace('Wyposażenie', '').str.strip()

In [100]:
##InformacjeDodatkowe
df_copy['InformacjeDodatkowe'] = df_copy['InformacjeDodatkowe'].str.replace('Informacje dodatkowe', '').str.strip()

In [101]:
##MaterialBudynku
df_copy['MaterialBudynku'] = df_copy['MaterialBudynku'].str.replace('Materiał budynku', '').str.strip()

In [102]:
df_copy.head()

,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,RodzajZabudowy,CzyWinda,Zabezpieczenia,Wyposazenie,InformacjeDodatkowe,MaterialBudynku,Wojewodztwo,Miasto,Powiat,Ulica
0,ID4hEoS,"Kawalerka 33m Balkon, nowe budownictwo, premium",9030.0,298000.0,"Skarżysko-Kamienna, skarżyski, świętokrzyskie",33.0,pełna własność,1,do zamieszkania,4/4,...,apartamentowiec,tak,"drzwi / okna antywłamaniowe, teren zamknięty, domofon / wideofon, monitoring / ochrona","zmywarka, lodówka, meble, piekarnik, kuchenka, telewizor, pralka",brak informacji,pustak,świętokrzyskie,Skarżysko-kamienna,Skarżyski,NaN
1,ID4qxDu,Komfortowy apartament Sopot Dolny,35070.0,2490000.0,"ul. Tadeusza Kościuszki, Centrum Południe, Dolny Sopot, Sopot, pomorskie",71.0,pełna własność,3,do zamieszkania,2/3,...,apartamentowiec,tak,"system alarmowy, teren zamknięty, domofon / wideofon, monitoring / ochrona","zmywarka, lodówka, meble, piekarnik, kuchenka, telewizor, pralka","klimatyzacja, piwnica",cegła,pomorskie,Sopot,Sopot,ul. Tadeusza Kościuszki
2,ID4qYhV,N/A,NaN,NaN,N/A,NaN,N/A,0,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,NaN,NaN,NaN,NaN
3,ID45Iqf,Mieszkanie - Sopot Centrum,12586.0,1100000.0,"Górny Sopot, Sopot, pomorskie",87.4,pełna własność,3,do zamieszkania,0/2,...,dom wolnostojący,nie,brak informacji,meble,brak informacji,brak informacji,pomorskie,Sopot,Sopot,NaN
4,ID45HQd,Mieszkanie 2 pokoje 38m2 Polesie,6959.0,270000.0,"Żubardź, Bałuty, Łódź, łódzkie",38.8,pełna własność,2,do remontu,8/10,...,brak informacji,tak,brak informacji,brak informacji,piwnica,brak informacji,łódzkie,Łódź,Łódź,NaN


In [103]:
df_copy['Zabezpieczenia'].value_counts()

N/A                                                                                       63
brak informacji                                                                           35
domofon / wideofon                                                                         9
monitoring / ochrona                                                                       5
teren zamknięty, domofon / wideofon                                                        5
drzwi / okna antywłamaniowe, teren zamknięty, domofon / wideofon, monitoring / ochrona     4
drzwi / okna antywłamaniowe, domofon / wideofon                                            2
system alarmowy                                                                            2
drzwi / okna antywłamaniowe, teren zamknięty, domofon / wideofon                           2
domofon / wideofon, monitoring / ochrona                                                   1
drzwi / okna antywłamaniowe, domofon / wideofon, monitoring / ochrona 

In [104]:
df_copy['Zabezpieczenia'] = df_copy['Zabezpieczenia'].replace('N/A', 'brak informacji')

In [105]:
df_copy['Zabezpieczenia'] = df_copy['Zabezpieczenia'].str.split(', ')
all_features = set(feature for sublist in df_copy['Zabezpieczenia'] for feature in sublist)

for feature in all_features:
    df_copy[feature] = df_copy['Zabezpieczenia'].apply(lambda x: 1 if feature in x else 0)
    

In [106]:
df_copy['Wyposazenie'] = df_copy['Wyposazenie'].replace('N/A', 'brak informacji')

In [107]:
df_copy['Wyposazenie'] = df_copy['Wyposazenie'].str.split(', ')
all_features = set(feature for sublist in df_copy['Wyposazenie'] for feature in sublist)

for feature in all_features:
    df_copy[feature] = df_copy['Wyposazenie'].apply(lambda x: 1 if feature in x else 0)

In [108]:
df_copy

,ID,Nazwa,CenaM2,Cena,Adres,Powierzchnia,Forma_Wlasnosci,LiczbaPokoi,StanWykonczenia,Pietro,...,rolety antywłamaniowe,system alarmowy,drzwi / okna antywłamaniowe,lodówka,piekarnik,meble,kuchenka,zmywarka,pralka,telewizor
0,ID4hEoS,"Kawalerka 33m Balkon, nowe budownictwo, premium",9030.0,298000.0,"Skarżysko-Kamienna, skarżyski, świętokrzyskie",33.0,pełna własność,1,do zamieszkania,4/4,...,0,0,1,1,1,1,1,1,1,1
1,ID4qxDu,Komfortowy apartament Sopot Dolny,35070.0,2490000.0,"ul. Tadeusza Kościuszki, Centrum Południe, Dolny Sopot, Sopot, pomorskie",71.0,pełna własność,3,do zamieszkania,2/3,...,0,1,0,1,1,1,1,1,1,1
2,ID4qYhV,N/A,NaN,NaN,N/A,NaN,N/A,0,N/A,N/A,...,0,0,0,0,0,0,0,0,0,0
3,ID45Iqf,Mieszkanie - Sopot Centrum,12586.0,1100000.0,"Górny Sopot, Sopot, pomorskie",87.4,pełna własność,3,do zamieszkania,0/2,...,0,0,0,0,0,1,0,0,0,0
4,ID45HQd,Mieszkanie 2 pokoje 38m2 Polesie,6959.0,270000.0,"Żubardź, Bałuty, Łódź, łódzkie",38.8,pełna własność,2,do remontu,8/10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,ID4qFqY,N/A,NaN,NaN,N/A,NaN,N/A,0,N/A,N/A,...,0,0,0,0,0,0,0,0,0,0
132,ID4qw1q,N/A,NaN,NaN,N/A,NaN,N/A,0,N/A,N/A,...,0,0,0,0,0,0,0,0,0,0
133,ID4qqjJ,Zamienię kawalerkę na działkę budowlaną,5181.0,100000.0,"ul. ks. Piotra Wawrzyniaka, Inowrocław, inowrocławski, kujawsko-pomorskie",19.3,pełna własność,1,Zapytaj,0,...,0,0,0,0,0,0,0,0,0,0
134,ID4qkjq,apartament SOPOT dolny 400 m do plaży i mola,21000.0,1050000.0,"ul. Fryderyka Chopina, Dolny Sopot - Grunwaldzka, Dolny Sopot, Sopot, pomorskie",50.0,pełna własność,3,do zamieszkania,2/2,...,0,0,1,1,1,1,1,1,0,1
